# Simple Conversational Agent
In this notebook, we will build a simple agent that is able to refer to the history of the interaction is order to provide relevant and succinct answers. It consists of:
* LLM client (LM Studio) to get answers from a large language model
* Customised prompt that consists of the user question and the conversation history
* History manager to manage the conversation history and context
* Message store to hold the messages for each conversation session.

As usual, we begin by loading the required packages and creating the LLM client. To set up LM Studio to serve a local model via an API, check out their guide [Run LM Studio as a service (headless)](https://lmstudio.ai/docs/app/api/headless).

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [ ]:
lm_studio_base_url = "http://localhost:1234/v1"
llm = ChatOpenAI(
    openai_api_base=lm_studio_base_url,
    max_tokens=1000,
    api_key="not-needed",
    model_name="gemma-3-4b-it",
    temperature=0.0,
)

In [ ]:
llm.invoke("What is the capital of France?")

Having verified that the LLM via LM Studio is responding correctly, we now create a simple chat history store which is kept in-memory.

In [ ]:
store = {}


def get_chat_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

Next, we create a prompt template that includes a system prompt, the conversation history, and the user's current question. To provide the conversation history, we include a placeholder which assumes that the history is in a dictionary with the specified key which is `history` in this context. To construct a prompt using this template, unless `optional` is set to `True`, it is assumed that the dictionary with the key is *already present*. But since the first message in the conversation has no history, it is empty.

In [ ]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful AI assistant."),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{input}"),
    ]
)

We now combine the history manager and prompt in a runnable chain i.e. we define a pipeline for how the functions should be executed.

In [ ]:
chain = prompt | llm

Finally, we wrap the chain, also called a *runnable* in Langchain parlance, in another runnable called `RunnableWithMessageHistory`. This class takes the runnable and the session history retrieval function as input. It then takes the responsibility of retrieving the history, given the session ID, of a given chat and updating it after every message.

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    runnable=chain,
    get_session_history=get_chat_history,
    input_message_key="input",
    history_message_key="history",
)

In [ ]:
session_id = "test_123"

response_1 = chain_with_history.invoke(
    {"input": "When was Karim Benzema born?"},
    config={
        "configurable": {"session_id": session_id},
    },
)
print(f"AI: {response_1.content}")

response_2 = chain_with_history.invoke(
    {"input": "Which club did he play the longest for?"},
    config={
        "configurable": {"session_id": session_id},
    },
)
print(f"AI: {response_2.content}")

In [ ]:
store["test_123"]